# Lab 6
Jakub Karbowski

# Wykorzystane biblioteki
- Clustering: algorytmy klasteryzacji i oceny jakości
- Distances: metryki

In [2]:
import Pkg
Pkg.add([
    "Clustering",
    "Distances",
])

using Clustering: hclust, cutree, silhouettes
using Distances: euclidean, cosine_dist
using Statistics: mean # <- to jest z std julii

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


# Preprocessing
Będę rozważał tylko litery a-z oraz spacje.
Usuwam więcej niż jedno wystąpienie spacji obok siebie.
Wszystkie znaki są też zamieniane na małe litery.

In [3]:
const PREPROCESS_CHARS = Tuple(['a':'z'; ' '])

('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ')

In [4]:
function preprocess(l)
    l = filter(lowercase(l)) do c
        c in PREPROCESS_CHARS
    end
    
    while true
        l1 = length(l)
        l = replace(l, "  " => " ")
        l2 = length(l)
        if l1 == l2
            break
        end
    end

    if l[1] == ' '
        l = l[2:end]
    end

    if l[end] == ' '
        l = l[1:end-1]
    end

    l
end

preprocess (generic function with 1 method)

# Stoplist
Usuwam 1% najczęstszych słów.

In [5]:
function applystoplist(lines)
    words = reduce(vcat, split.(lines)) |> unique
    counts = zeros(Int, length(words))
    for l = lines, w = split(l)
        i = argmax(i -> words[i] == w, eachindex(words))
        counts[i] += 1
    end
    wc = zip(words, counts) |> collect
    sort!(wc, by=(wc -> wc[2]), rev=true)
    stoplist = getindex.(wc, 1)[1:div(end, 100)]
    @show stoplist
    map(lines) do l
        join(filter(w -> !(w in stoplist), split(l, ' ')), ' ')
    end
end

applystoplist (generic function with 1 method)

# Wektoryzacja
Najprostsza, zliczam wystąpienia każdej litery w wektorze długości 27 (26 liter + spacja).

In [6]:
function vectorize(l)
    [count(s -> s == c, l) for c = PREPROCESS_CHARS]
end

vectorize (generic function with 1 method)

# Ładowanie danych

In [7]:
loadlines(n) = collect(
    Iterators.filter(
        l -> !isempty(l),
        Iterators.take(
            readlines("data/lines.txt"),
            n,
        ),
    )
)

loadlines (generic function with 1 method)

# Levenshtein
Można było lepiej bez macierzy $n^2$.

In [8]:
function levenshtein(t1, t2)
    delta(x, y) = x == y ? 0 : 1
    
    n1 = length(t1)
    n2 = length(t2)

    f = zeros(Int, n1 + 1, n2 + 1)
    f[1:end, 1] = 0:n1
    f[1, 1:end] = 0:n2
    
    for i = 2:n1+1, j = 2:n2+1
        up = f[i-1, j] + 1
        left = f[i, j-1] + 1
        upleft = f[i-1, j-1] + delta(t1[i-1], t2[j-1])

        f[i, j] = min(up, left, upleft)
    end
    
    f[end, end]
end

levenshtein("levenshtein", "lewensztain")

3

# Klasteryzacja
Używam hierarchical clustering. Drzewo ucinane jest na takiej wysokości,
aby uzyskać "rozsądną" liczbę klastrów.

Testując różne metryki, staram się zachować taką samą liczbę klastrów.

## Ocena jakości
Do oceny jakości klastrów, stosuję funkcję [silhouettes](https://en.wikipedia.org/wiki/Silhouette_(clustering)).
Biorę średnią wyników dla wszystkich punktów.
Większa wartość oznacza lepszą klasteryzację.

In [9]:
function runclustering(lines, dist, h)
    d = [dist(a, b) for a = lines, b = lines]

    tree = hclust(d)
    assignments = cutree(tree, h=h)
    clusters = [Int[] for i = 1:maximum(assignments)]
    for p = eachindex(lines)
        push!(clusters[assignments[p]], p)
    end
    counts = length.(clusters)

    println("Number of clusters: $(length(clusters))")

    quality = mean(silhouettes(assignments, counts, d))
    println("Quality: $quality")

    println("Biggest cluster:")
    c = argmax(counts)
    [lines[i] for i = clusters[c]] .|> println

    quality
end

runclustering (generic function with 1 method)

# Levenshtein, no stop list

In [10]:
lines = loadlines(700) .|> preprocess
levnostop = runclustering(lines, levenshtein, 5)
;

Number of clusters: 497
Quality: 0.3746279170467211
Biggest cluster:
actona company assmedegaardvej tvis holstebrodenmark
actona company assmedegaardvej tvis holstebrodenmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej atvis holstebrodenmark
actona company as smedegardsvej a tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej a tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark


# Levenshtein, stop list

In [11]:
lines = loadlines(700) .|> preprocess |> applystoplist
levstop = runclustering(lines, levenshtein, 5)
;

stoplist = SubString{String}["tel", "fax", "ltd", "poland", "russia", "str", "ul", "gdynia", "a", "llc", "sa", "telfax", "moscow", "oo", "forwarding", "sp", "logistics", "china", "petersburg"]
Number of clusters: 504
Quality: 0.3667066576704834
Biggest cluster:
actona company assmedegaardvej tvis holstebrodenmark
actona company assmedegaardvej tvis holstebrodenmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej atvis holstebrodenmark
actona company as smedegardsvej tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark


Powstały sensowne klastry. Gorsza jakość ze stoplistą.

# Euclidean, no stop list

In [12]:
lines = loadlines(700) .|> preprocess
eucnostop = runclustering(lines, (a, b) -> euclidean(a |> vectorize, b |> vectorize), 2.3)
;

Number of clusters: 505
Quality: 0.30593071258029886
Biggest cluster:
actona company assmedegaardvej tvis holstebrodenmark
actona company assmedegaardvej tvis holstebrodenmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej atvis holstebrodenmark
actona company as smedegardsvej a tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej a tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark


# Euclidean, stop list

In [13]:
lines = loadlines(700) .|> preprocess |> applystoplist
eucstop = runclustering(lines, (a, b) -> euclidean(a |> vectorize, b |> vectorize), 2.3)
;

stoplist = SubString{String}["tel", "fax", "ltd", "poland", "russia", "str", "ul", "gdynia", "a", "llc", "sa", "telfax", "moscow", "oo", "forwarding", "sp", "logistics", "china", "petersburg"]
Number of clusters: 511
Quality: 0.3213001927263695
Biggest cluster:
actona company assmedegaardvej tvis holstebrodenmark
actona company assmedegaardvej tvis holstebrodenmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej tvis dk holstebro
actona company as smedegaardvej atvis holstebrodenmark
actona company as smedegardsvej tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark


Gorsza jakość niż Levenshtein, szybsze obliczenia. Ze stoplistą lepiej niż bez.

# Cosine, no stop

In [14]:
lines = loadlines(700) .|> preprocess
cosnostop = runclustering(lines, (a, b) -> cosine_dist(a |> vectorize, b |> vectorize), 0.006)
;

Number of clusters: 501
Quality: 0.38935936246229996
Biggest cluster:
actona company assmedegaardvej tvis holstebrodenmark
actona company assmedegaardvej tvis holstebrodenmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej atvis holstebrodenmark
actona company as smedegardsvej a tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej a tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark


# Cosine, stop

In [15]:
lines = loadlines(700) .|> preprocess |> applystoplist
cosstop = runclustering(lines, (a, b) -> cosine_dist(a |> vectorize, b |> vectorize), 0.006)
;

stoplist = SubString{String}["tel", "fax", "ltd", "poland", "russia", "str", "ul", "gdynia", "a", "llc", "sa", "telfax", "moscow", "oo", "forwarding", "sp", "logistics", "china", "petersburg"]
Number of clusters: 534
Quality: 0.3355471405562397
Biggest cluster:
actona company assmedegaardvej tvis holstebrodenmark
actona company assmedegaardvej tvis holstebrodenmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej tvis holstebro denmark
actona company as smedegaardvej atvis holstebrodenmark
actona company as smedegardsvej tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark
actona company as smedegardvej tvis dk holstebro denmark
actona company as smedegardvej atvis dk holstebro denmark


Lepsza jakość niż Levenshtein, jeszcze szybsze obliczenia.
Gorzej ze stoplistą niż bez.

# Podsumowanie

In [16]:
[
    "" "Levenshtein" "Euclidean" "Cosine"
    "No stoplist" levnostop eucnostop cosnostop
    "Stoplist" levstop eucstop cosstop
]

3×4 Matrix{Any}:
 ""              "Levenshtein"   "Euclidean"   "Cosine"
 "No stoplist"  0.374628        0.305931      0.389359
 "Stoplist"     0.366707        0.3213        0.335547

Najlepsze metryki:
- Cosinusowa bez stoplisty
- Levenshteina bez stoplisty

Stoplista polepsza klasteryzację tylko dla metryki euklidesowej.

# Teraz robi się ciekawie
Flux może się trochę długo instalować.

In [18]:
import Pkg
Pkg.add([
    "Flux",
    "BSON",
])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [19]:
using Flux
using Flux: DataLoader, params, onecold, onehot, Embedding, Recur, LSTMCell, throttle, @epochs
using Flux.Losses: logitcrossentropy
using Random: shuffle!, seed!
using BSON: @save, @load
using Downloads: download

# Generowanie danych uczących
Biorę pod uwagę tylko litery a-z oraz '_'.

Litery są one-hot kodowane.

Napisy są zapisywane jako ciągi wektorów one-hot.

Sieć będzie uczona na parach sekwencji:
- wejście: dwa napisy
- wyjście: czy napisy należą do tego samego klastra (dwie klasy TAK/NIE zapisane one-hot)

Wykorzystałem plik clusters.txt z przykładową klasteryzacją.

Tworząc zbiór uczący, należy zapewnić równą liczbę
przykładów dla obu klas.

Dla każdego napisu biorę $n$ napisów z klastra tego napisu
jako przykłady TAK, oraz $n$ losowych napisów z innych klastrów
jako przykłady NIE.

Dzięki temu mamy 50% przykładów TAK i 50% NIE.

Dla uproszczenia sieci zakładam, że porównywane napisy mają taką samą długość.
W tym celu dodaję na końcu padding '_' do krótszego napisu w każdej parze osobno.

Sieć jest rekurencyjna, więc może działać na napisach dowolnej długości,
jedynie długości porównywanych napisów muszą być takie same.

In [22]:
const CHARACTERS = Tuple([('a':'z'); '_'])

function loadclusters()
    clusters = Vector{Vector{String}}([[]])
    for l = filter(l -> length(l) > 0, readlines("data/clusters.txt"))
        if count(c -> c == '#', l) == length(l)
            push!(clusters, [])
        else
            push!(clusters[end], l)
        end
    end
    filter(c -> !isempty(c), clusters)
end

function encodelines(l1, l2)
    l1 = filter(c -> c in CHARACTERS, lowercase(l1))
    l2 = filter(c -> c in CHARACTERS, lowercase(l2))

    l = max(length(l1), length(l2))

    @assert l > 0

    l1 *= repeat('_', l - length(l1))
    l2 *= repeat('_', l - length(l2))

    [(onehot(c1, CHARACTERS), onehot(c2, CHARACTERS)) for (c1, c2) = zip(l1, l2)]
end

function encodedata(clusters::Vector{Vector{String}})
    T = typeof(onehot(CHARACTERS[1], CHARACTERS))
    xs = Vector{Vector{Tuple{T, T}}}()
    ys = Vector{typeof(onehot(false, (false, true)))}()

    for (c1i, c1) = enumerate(clusters)
        for l1 = c1
            for l2 = c1
                push!(xs, encodelines(l1, l2))
                push!(ys, onehot(true, (false, true)))
            end
            i = 0
            while i < length(c1)
                c2i = rand(1:length(clusters))
                if c2i == c1i
                    continue
                end
                
                c2 = clusters[c2i]
                l2 = rand(c2)
                
                push!(xs, encodelines(l1, l2))
                push!(ys, onehot(false, (false, true)))
                
                i += 1
            end
        end
    end

    xs, ys
end

encodedata (generic function with 1 method)

# Model
Model posiada:
- 2 takie same warstwy Embedding do zamiany wektorów one-hot znaków na 16 wymiarowe wektory.
- 2 takie same warstwy LSTM do wstępnej analizy obu napisów niezależnie.
- 1 warstwę LSTM do porównywania napisów.
- Warstwę wyjściową (2 neurony - 2 klasy).

```
Sequence 1 --> Embedding 1 --> LSTM 1 --\
                                         |--> LSTM 3 --> Output
Sequence 2 --> Embedding 2 --> LSTM 2 --/
```

Warstwy Embedding 1/2 oraz LSTM 1/2 współdzielą wagi.

In [23]:
struct SemSim{E, L, R, C}
    embedder::E
    extractor::L
    extractor1::R
    extractor2::R
    comparator::C
end

function SemSim()
    extractor = LSTMCell(16 => 16)
    SemSim(
        Embedding(length(CHARACTERS) => 16),
        extractor,
        Recur(extractor),
        Recur(extractor),
        Chain(
            LSTM(32 => 16),
            Dense(16 => 2),
        ),
    )
end

function (m::SemSim)(x::T) where T <: Tuple
    m.comparator(vcat(
        m.extractor1(m.embedder(x[1])),
        m.extractor2(m.embedder(x[2])),
        ))
end

function (m::SemSim)(xs::T) where T <: Vector
    Flux.reset!(m)
    y = m(xs[1])
    for i = 2:length(xs)
        y = m(xs[i])
    end
    y
end

classify(m, l1, l2) = onecold(m(encodelines(l1, l2)), (false, true))

function Flux.reset!(m::SemSim)
    Flux.reset!(m.extractor1)
    Flux.reset!(m.extractor2)
    Flux.reset!(m.comparator)
end

Flux.@functor SemSim (embedder, extractor, comparator)

# Uczenie (dla cierpliwych)
Potrzeba ~15 min na CPU (MacBook M1).

Proces uczenia będzie co jakiś czas zapisywał model do pliku.

Gradient liczony jest na batchach po 100 przykładów.

Dodałem regularyzację wag L2.

In [28]:
function train(traindata, testdata)
    traindata = DataLoader(traindata, shuffle=true, batchsize=100)
    testdata = zip(testdata...) |> collect
    
    m = SemSim() |> gpu
    ps = params(m)

    sqnorm(x) = sum(abs2, x)
    penalty() = sum(sqnorm, ps) / 20000
    loss(xs, ys) = sum(logitcrossentropy(m(x), y) for (x, y) = zip(xs, ys)) / length(xs) + penalty()
    
    opt = ADAM(0.01)
    
    function eval(dat)
        @info "Evaluating..."
        good = 0
        for (x, y) = dat
            yp = m(x)
            if onecold(yp) == onecold(y)
                good += 1
            end
        end
        acc = good / length(dat) * 100
        @info "acc = $acc%"

        @save "data/semsim-$(round(Int, acc)).bson" m opt
    end

    evalcb = throttle(() -> eval(rand(testdata, 2000)), 60)

    while true
        Flux.train!(loss, ps, traindata, opt, cb = evalcb)
    end
end

train (generic function with 1 method)

# Dane uczące/testujące
1. Ładuję klastry.
2. Wybieram 25% klastrów jako dane testujące.
3. Niezależnie dla klastrów uczących i testujących tworzę pary napisów.

Dzięki temu mamy całkowicie niezależne dane uczące i testujące.

W każdym zbiorze mamy równy rozkład przykładów dla każdej klasy.

In [29]:
seed!(42)

clusters = loadclusters()
shuffle!(clusters)
testclusters = clusters[1:div(end, 4)]
trainclusters = clusters[div(end, 4) + 1 : end]

traindata = encodedata(trainclusters)
testdata = encodedata(testclusters)

length.((traindata[2], testdata[2]))

(101254, 41828)

Dla odważnych. Można włączyć uczenie.

In [30]:
# train(traindata, testdata)

Lepiej jednak wykorzystać model, który już nauczyłem.

In [31]:
mpath = "data/semsim-95.bson"
@load mpath m opt

# Ocena dokładności modelu
Uruchamiamy model na danych testujących.
Trochę to zajmuje czasu.

In [32]:
begin
    good = 0
    data = zip(testdata...) |> collect
    for (x, y) = data
        yp = m(x)
        if onecold(yp) == onecold(y)
            good += 1
        end
    end
    acc = round(good / length(data) * 100, digits=2)
    @info "acc = $acc%"
end

┌ Info: acc = 95.0%
└ @ Main In[32]:11


Dokładność: 95%

Brzmi aż za dobrze. Jednak faktycznie, model nauczył
się tak dobrze klasyfikować dane, których nigdy nie widział.

Czy na pewno jest tak "mądry"?

In [33]:
classify(m, "Politechnika Krakowska", "Krakowska Politechnika")

false

In [34]:
classify(m, "Politechnika Krakowska", "olitechnika Krakowska")

false

In [35]:
classify(m, "Politechnika Krakowska", "Bolitechnika Krakowska")

true

In [36]:
classify(m, "Politechnika Krakowska", "Palitechnika Krakowska")

true

In [37]:
classify(m, "Politechnika Krakowska", "Politechnika")

true

In [38]:
classify(m, "Politechnika Krakowska", "Politec Gdanska")

true

In [39]:
classify(m, "Kramowska", "Krakowska")

true

In [40]:
classify(m, "Kakowska", "Krakowska")

false

Widać, że model głównie patrzy na kilka pierwszych znaków.

Najwyraźniej dla danych jakie miałem, wystarcza to do osiągnięcia takiej dokładności.

Bez trudniejszych przykładów w danych nie nauczymy lepszego modelu.

Jednak jeśli kryterium jakości jest klasteryzacja pliku clusters.txt,
model sprawdza się bardzo dobrze.